In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

## Export Json Files - Json_Award Clean Up 

In [27]:
#Import Json File for Json_Awards and Clean up/Normalize Data 
import json
json_file = "Resources/json_award.json"
with open(json_file, 'r') as f:
    data = json.loads(f.read())
award_df =  pd.json_normalize(data)
award_df["laureates"] = award_df["laureates"].str[0]
award_df["laureates"].apply(pd.Series)
#Note id, award year, prize amount, prize amount adjusted, category, are all important categories in this dataset.  
#I think we should likely focus on these and drop "known name" since that's what were using to join from our other table
award_df = pd.concat([award_df.drop(["laureates"],axis=1),award_df["laureates"].apply(pd.Series)],axis=1)
award_df.head()

award_df = award_df[['id', 'awardYear', 'prizeAmount', 'categoryFullName.en' ]]
award_df

award_df = award_df.rename(columns = {'id': 'laureate_id','awardYear' : 'year_awarded', 'prizeAmount' : 'prize_amount', 'categoryFullName.en': 'nobel_prize'})
award_df





,laureate_id,year_awarded,prize_amount,nobel_prize
0,160,1901,150782,The Nobel Prize in Chemistry
1,569,1901,150782,The Nobel Prize in Literature
2,462,1901,150782,The Nobel Peace Prize
3,1,1901,150782,The Nobel Prize in Physics
4,293,1901,150782,The Nobel Prize in Physiology or Medicine
...,...,...,...,...
641,982,2019,9000000,The Sveriges Riksbank Prize in Economic Scienc...
642,980,2019,9000000,The Nobel Prize in Literature
643,981,2019,9000000,The Nobel Peace Prize
644,973,2019,9000000,The Nobel Prize in Physics


## Export Json_Laureates

In [24]:
#Export json file and normalize the data 
json_file = "Resources/json_laureates.json"
with open(json_file, 'r') as f:
    data = json.loads(f.read())
l_df =  pd.json_normalize(data)
l_df

,laureate_id,recipient,gender,birth_date,birth_place
0,745,A. Michael Spence,male,1943-00-00,USA
1,102,Aage N. Bohr,male,1922-06-19,Denmark
2,779,Aaron Ciechanover,male,1947-10-01,British Protectorate of Palestine
3,259,Aaron Klug,male,1926-08-11,Lithuania
4,114,Abdus Salam,male,1926-01-29,India
...,...,...,...,...,...
938,826,Yoichiro Nambu,male,1921-01-18,Japan
939,927,Yoshinori Ohsumi,male,1945-02-09,Japan
940,265,Yuan T. Lee,male,1936-11-19,Taiwan
941,794,Yves Chauvin,male,1930-10-10,Belgium


## Translate Laureates Table for SQL loading

In [ ]:
# We filled in the known name column to contain both the Laureates names that were people/organizations
l_df['knownName.en'].fillna(l_df["orgName.en"], inplace=True)
l_df['knownName.en'].count()

#Select the columns that we would like to transfer over into our SQL table 
l_df = l_df[['id','knownName.en','gender','birth.date','birth.place.country.en']]

#Rename the columns as what's available in the SQL table 
l_df = l_df.rename(columns = {'id' : 'laureate_id', 'knownName.en' : 'recipient', 'birth.date' : 'birth_date', 'birth.place.country.en': 'birth_place', 'gender' : 'gender'})
l_df

## Load Data into pgAdmin/SQL

In [11]:
rds_connection_string = "postgres:postgres@localhost:5432/Nobel_Prize_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
#check for available tables
engine.table_names()

['laureates', 'award']

In [22]:
l_df.to_sql(name='laureates', con=engine, if_exists='append', index=False)

In [28]:
award_df.to_sql(name='award', con=engine, if_exists='append', index=False)